In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
from open_experiment import control_deaccu,fixedSM_deaccu
import object_detection as obj

In [3]:
import tobac
print(tobac.__version__)

1.4.2


In [4]:
# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [5]:
def update_lon(ds,longitude='lon'):
    ds.coords[longitude] = (ds.coords[longitude] + 180) % 360 - 180
    ds = ds.sortby(ds.lon)
    return ds

In [6]:
path='/scratch/wcq7pz/exp_levante_post/'
## open topography, land_fraction
topo5km = xr.open_dataset(path+'topography_dom03_5km.nc')
topo5km.coords['lon'] = (topo5km.coords['lon'] + 180) % 360 - 180
topo5km  = topo5km.sortby(topo5km.lon)

## READ AMAZON MASK
onlyab = xr.open_dataset('onlyab5km.nc')
maskAB = onlyab.interp(lat = topo5km.lat,lon = topo5km.lon)

### Define functions

In [7]:
def to_dataset(array,xarr,time=True,varname='mask'):
    if time==True:
        ds = xr.Dataset( { varname: (["time","lat", "lon"], array)},
    coords={ "time":(["time"],xarr.time.values), "lat": (["lat"], xarr.lat.values),"lon": (["lon"], 
             xarr.lon.values)})
    else:
        ds = xr.Dataset( { varname: (["lat", "lon"], array)},
    coords={ "lat": (["lat"], xarr.lat.values),"lon": (["lon"], xarr.lon.values)}) 
    
    return ds

In [8]:
def sel_by_lifetime(Track,hours=3,operator='>'):
    """
    for hourly data
    """
    counts = Track.groupby("cell")["time_cell"].count().values
    if operator == '<':
        counts_min = Track.groupby("cell")["time_cell"].count()[counts<hours]
    elif operator == '>':
        counts_min = Track.groupby("cell")["time_cell"].count()[counts>hours]
    else:
        raise ValueError('Invalid operator. Choose either "<" or ">".')
    selected_cells = Track[Track["cell"].isin(counts_min.reset_index()[counts_min.reset_index().cell>0].cell)]
    selected_cells = selected_cells.reset_index()
    #selected_cells = selected_cells.rename(columns={'lon': 'longitude'})
    return selected_cells

In [9]:
def subset_mask_new(df,ds_mask,var='labels_ocs',var2='mask_obs'):
    # Filter the original dataset to only include times that are in the dataframe
    original_ds_filtered = ds_mask.sel(time=df['time'].unique())
    #new_mask = np.zeros_like(original_ds_filtered.labels_ocs);
    
    #create boolean
    new_mask = [np.isin(original_ds_filtered.sel(time=i)[var],
                        df[df.time==i].idxn.values) for i in original_ds_filtered.time.values] 
    
    #make boolean a dataset
    ds_new_mask = to_dataset(new_mask,original_ds_filtered)
    
    #apply to ds_filtered
    new_ds = original_ds_filtered[var2].where(ds_new_mask.mask==True)
        

    return(new_ds)  

In [10]:
def update_lon_attrs(ds):
    ds.lon.attrs["standard_name"] = "longitude"
    ds.lon.attrs["long_name"] = "longitude"
    ds.lon.attrs["units"] = "degrees_east"
    ds.lon.attrs["axis"] = "X"
    return ds

In [11]:
control_deaccu = update_lon_attrs(control_deaccu); 
fixedSM_deaccu = update_lon_attrs(fixedSM_deaccu)


## Load objects

In [13]:
## Open objects (dataframes) and masks (3D datasets)
## *************** Amazon region  ***************
ds_ob_control_AB = xr.open_dataset('ds_ob_control5k_Amazon.nc') 
ds_ob_fixedSM_AB = xr.open_dataset('ds_ob_fixedSM5k_Amazon.nc')

df_c_AB = pd.read_pickle('pkl_files/df_ob_control5k_Amazon.pkl') 
df_f_AB = pd.read_pickle('pkl_files/df_ob_fixedSM5k_Amazon.pkl') 

## *************** SESA region  ***************
ds_ob_control_SESA = xr.open_dataset('ds_ob_control5k_SESA.nc') 
ds_ob_fixedSM_SESA = xr.open_dataset('ds_ob_fixedSM5k_SESA.nc')

df_c_SESA = pd.read_pickle('pkl_files/df_ob_control5k_SESA.pkl') 
df_f_SESA = pd.read_pickle('pkl_files/df_ob_fixedSM5k_SESA.pkl')

In [14]:
#Rename columns to prepare for TOBAC tracking

## *************** Amazon region  ***************
df_c_AB = df_c_AB.reset_index(drop=True).reset_index().rename(columns={'index': 'feature'})
df_c_AB['feature'] += 1 # Add 1 to the values in the "feature" column
df_c_AB = df_c_AB.rename(columns={"y": "hdim_1", "x": "hdim_2"})

df_f_AB = df_f_AB.reset_index(drop=True).reset_index().rename(columns={'index': 'feature'})
df_f_AB['feature'] += 1
df_f_AB = df_f_AB.rename(columns={"y": "hdim_1", "x": "hdim_2"})

## *************** SESA region  ***************
df_c_SESA = df_c_SESA.reset_index(drop=True).reset_index().rename(columns={'index': 'feature'})
df_c_SESA['feature'] += 1
df_c_SESA = df_c_SESA.rename(columns={"y": "hdim_1", "x": "hdim_2"})

df_f_SESA = df_f_SESA.reset_index(drop=True).reset_index().rename(columns={'index': 'feature'})
df_f_SESA['feature'] += 1
df_f_SESA = df_f_SESA.rename(columns={"y": "hdim_1", "x": "hdim_2"})

In [15]:
# Create the "frame" column by grouping the data by unique values of "time"
df_c_AB['frame'] = df_c_AB.groupby('time').ngroup()
df_f_AB['frame'] = df_f_AB.groupby('time').ngroup()

df_c_SESA['frame'] = df_c_SESA.groupby('time').ngroup()
df_f_SESA['frame'] = df_f_SESA.groupby('time').ngroup()

## Tracking

In [16]:
# Grid spacing of the input data (in meter)
dxy = 4950 #(5km)
dt = 3600 #200

In [17]:
# Dictionary containing keyword arguments for the linking step:
parameters_linking={}
parameters_linking['method_linking']='predict'
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['subnetwork_size']=100
parameters_linking['memory']=0
parameters_linking['time_cell_min']=5*60
parameters_linking['v_max']=10 

In [18]:
Track_c_AB = tobac.linking_trackpy(df_c_AB,control_deaccu.tot_prec,dt=dt,dxy=dxy,**parameters_linking);
Track_f_AB = tobac.linking_trackpy(df_f_AB,fixedSM_deaccu.tot_prec,dt=dt,dxy=dxy,**parameters_linking);

Track_c_SESA = tobac.linking_trackpy(df_c_SESA,control_deaccu.tot_prec,dt=dt,dxy=dxy,**parameters_linking);
Track_f_SESA = tobac.linking_trackpy(df_f_SESA,fixedSM_deaccu.tot_prec,dt=dt,dxy=dxy,**parameters_linking);

Frame 847: 4 trajectories present.


## Lifetime

In [19]:
## Select objects with a lifetime larger than 3 hours
sel_cells_c_AB = sel_by_lifetime(Track_c_AB);
sel_cells_f_AB = sel_by_lifetime(Track_f_AB); 

sel_cells_c_SESA = sel_by_lifetime(Track_c_SESA);
sel_cells_f_SESA = sel_by_lifetime(Track_f_SESA); 

In [21]:
print(len(sel_cells_c_AB.cell.unique()),len(sel_cells_f_AB.cell.unique())) 

print(len(sel_cells_c_SESA.cell.unique()),len(sel_cells_f_SESA.cell.unique()))

1145 1072
139 162


In [23]:
## Prepare column 'idxn' to identify labels in the datasets (>3h)
sel_cells_c_AB['idxn'] = (pd.to_numeric(sel_cells_c_AB['idx'],errors = 'coerce'))
sel_cells_f_AB['idxn'] = (pd.to_numeric(sel_cells_f_AB['idx'],errors = 'coerce'))

sel_cells_c_SESA['idxn'] = (pd.to_numeric(sel_cells_c_SESA['idx'],errors = 'coerce'))
sel_cells_f_SESA['idxn'] = (pd.to_numeric(sel_cells_f_SESA['idx'],errors = 'coerce'))

In [25]:
## new_mask of OCS (objects lasting > 3h)  
nMask_c_AB = subset_mask_new(sel_cells_c_AB,ds_ob_control_AB)
nMask_f_AB = subset_mask_new(sel_cells_f_AB,ds_ob_fixedSM_AB)

nMask_c_SESA = subset_mask_new(sel_cells_c_SESA,ds_ob_control_SESA)
nMask_f_SESA = subset_mask_new(sel_cells_f_SESA,ds_ob_fixedSM_SESA)

In [36]:
## Save mask of OCS (objects lasting > 3h)  
nMask_c_AB.to_netcdf('nMask_control_ocs_AB.nc'); 
nMask_f_AB.to_netcdf('nMask_fixedSM_ocs_AB.nc')

nMask_c_SESA.to_netcdf('nMask_control_ocs_SESA.nc'); 
nMask_f_SESA.to_netcdf('nMask_fixedSM_ocs_SESA.nc')

#### Add stage information

In [27]:
def add_stage_convective_system(df_tracked):
    """
    Two main stages are initial t_i and t_f correspondent to first and last timesteps. Intermediate stages
    are denoted t_1, t_2,..,t_n
    """
    df = df_tracked[['cell','time_cell']].copy()

    # count the number of stages for each cell
    n_stages = df.groupby('cell')['time_cell'].count() - 1

    # create a dictionary of stages for each cell
    stages_dict = {}
    for cell in n_stages.index:
        if n_stages.loc[cell] == 2:
            stages_dict[cell] = ['t_i'] + ['t_m'] + ['t_f']
        elif (n_stages.loc[cell] > 2) and (n_stages.loc[cell] < 7):
            stages_dict[cell] = ['t_i'] * 2 + ['t_m'] * (n_stages[cell]-3) + ['t_f'] * 2
        
        else:            
            stages_dict[cell] = ['t_i'] * 3 + ['t_m'] * (n_stages[cell]-5) + ['t_f'] * 3

    # create a new column 'stage' based on the cell and the stages dictionary
    df['stage'] = [stages_dict[df.loc[i, 'cell']][df.groupby('cell').cumcount()[i]] for i in range(len(df))]
    
    dfn = df_tracked.reset_index(drop=True);
    dfn['stage'] = df['stage']

    return(dfn)

In [28]:
## Select objects with a lifetime larger than 3 hours
df_stage_c_AB = add_stage_convective_system(sel_cells_c_AB)
df_stage_f_AB = add_stage_convective_system(sel_cells_f_AB);

df_stage_c_SESA = add_stage_convective_system(sel_cells_c_SESA)
df_stage_f_SESA = add_stage_convective_system(sel_cells_f_SESA)

In [30]:
df_stage_c_AB.to_pickle('pkl_files/df_stage_c_gt3h_AB.pkl');
df_stage_f_AB.to_pickle('pkl_files/df_stage_f_gt3h_AB.pkl')

df_stage_c_SESA.to_pickle('pkl_files/df_stage_c_gt3h_SESA.pkl');
df_stage_f_SESA.to_pickle('pkl_files/df_stage_f_gt3h_SESA.pkl')